In [10]:
import pandas as pd
import datetime
import itertools
import requests
from bs4 import BeautifulSoup
import re

In [59]:
def scrape_shows(dir_url):
    url_beginning = dir_url + str(1)
    hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

    # Request page content
    dir_page_response = requests.get(url_beginning, timeout=120, headers = hdr)

    # Parse and turn into soup
    dir_page_soup = BeautifulSoup(dir_page_response.content, 'html.parser')

    max_seasons = dir_page_soup.find_all(class_ = 'current')
    max_seasons = list(str(max_seasons[0].text).strip())
    max_seasons = [y.strip('\n') for y in max_seasons]
    max_seasons = [y.strip(' ') for y in max_seasons]

    all_seasons = []
    for i in max_seasons:
        if bool(str(i)) == True:
            all_seasons.append(i)
        else:
            continue

    season_length = len(all_seasons) + 1


    name_of_episode = []
    season_episode = []
    get_ratings = []

    for i in range(1, season_length):
        url = dir_url + str(i)
        dir_page_response = requests.get(url, timeout=120, headers = hdr)
        dir_page_soup = BeautifulSoup(dir_page_response.content, 'html.parser')
        episodes = dir_page_soup.find_all(class_ = 'image')
        max_episodes_per_season = len(episodes)

        for i in range(0, max_episodes_per_season):
            episode_name = str(episodes[i].contents[1]).split('title')[2].split('<div')[0]
            name_of_episode.append(episode_name)

        for i in range(0, max_episodes_per_season):
            season_and_episode = episodes[i].contents[1].text.strip()
            season_episode.append(season_and_episode)
            
        try:
            for i in range(0, max_episodes_per_season):
                ratings = dir_page_soup.find_all(class_ = 'ipl-rating-widget')
                rating = str(ratings[i].contents[1].text.strip()).split()[0]
                get_ratings.append(rating)
        except:
            continue
            
    
    # show_name
    show = dir_page_soup.find_all(class_ = 'parent')
    show_name = [show[0].contents[1].text.strip().split('\n')[0].strip()] * len(name_of_episode)
    
                    
    # final cleaning
    name_of_episode = [y.strip('=">') for y in name_of_episode]
    name_of_episode = list(map( lambda x: x.replace(">", ''), name_of_episode))
    name_of_episode = list(map( lambda x: x.replace('" ', ''), name_of_episode))

    df = pd.DataFrame(list(zip(season_episode, name_of_episode, get_ratings, show_name)))
    df.columns = ['season_episode', 'title', 'ratings', 'show_name']
    df_split = df['season_episode'].str.split(',', expand = True)
    result = pd.concat([df, df_split], axis=1, sort=False)
    result.columns = ['remove', 'title', 'rating', 'show_name', 'season', 'episode']
    df = result[['title', 'rating', 'show_name','season', 'episode']]
    
    return df

In [60]:
df = scrape_shows(dir_url = 'https://www.imdb.com/title/tt3107288/episodes?season=')